# Tutorial 2: The Classification Hierarchy
## Pre-orders as Categories

---

### A Note from the Archive Review Committee

*To the research assistant:*

*Your analysis of Vold's Passage Diagrams has proven illuminating. The Committee now asks you to examine her secondary claim—that the Archive's classification system itself has categorical structure.*

*The Archive organizes documents, creatures, and places into hierarchies: scholarly works contain mathematical treatises; creatures contain predators; predators contain apex predators. Vold noticed that these containment relationships have a peculiar property: between any two categories, there is at most ONE way to go from one to the other.*

*This is unlike the passage diagrams, where multiple morphisms might connect the same pair of objects (different creatures taking different paths). In a classification hierarchy, either A is contained in B, or it isn't. There's no "multiple ways" to be contained.*

*Vold wrote: "The Archive itself is a category. Every classification is a morphism. To place a document is to draw an arrow. But unlike the passages of creatures, these arrows are unique—or absent."*

*Your task: Verify Vold's claim by examining the Archive's classification records. Determine whether hierarchical classification forms a category, and if so, what special properties it possesses.*

—*Archive Review Committee, Year 934*

---

## What You Will Learn

In this tutorial, you will learn to:

1. Load and explore the Archive's classification data
2. Understand **pre-orders** as special categories
3. Identify **reflexivity** and **transitivity** in hierarchical data
4. Distinguish between **pre-orders** and **partial orders**
5. Build and visualize hierarchical categories

By the end, you will understand:
- How classification hierarchies have categorical structure
- Why "at most one morphism" defines a pre-order category
- How taxonomies, organizational charts, and file systems are all pre-order categories

---

## The Mathematical Background

In Tutorial 1, we saw that categories can have multiple morphisms between the same pair of objects. A stakdur might take several different paths from its territory to the reed marsh, and each path is a distinct morphism.

But some categories are more restrictive. A **pre-order category** has a special property:

> Between any two objects A and B, there is **at most one** morphism from A to B.

In notation: |Hom(A, B)| ≤ 1 for all objects A, B.

This means we can replace morphisms with a simple relation: either there EXISTS a morphism A → B (written A ≤ B), or there doesn't.

### Pre-orders in Everyday Life

| Structure | Objects | "≤" Relation |
|-----------|---------|-------------|
| File system | Directories | "is a subdirectory of" |
| Taxonomy | Categories | "is a subcategory of" |
| Org chart | Positions | "reports to" |
| Family tree | Individuals | "is a descendant of" |
| Subsets | Sets | "is contained in" |

### Properties of Pre-orders

For a relation ≤ to be a pre-order, it must be:

1. **Reflexive**: A ≤ A for all A (identity morphisms exist)
2. **Transitive**: If A ≤ B and B ≤ C, then A ≤ C (composition exists)

These are exactly the category axioms! A pre-order IS a category where morphisms are unique.

### Partial Orders

A **partial order** adds one more property:

3. **Antisymmetric**: If A ≤ B and B ≤ A, then A = B

This means no two distinct objects can each contain the other. Most classification systems are partial orders, not just pre-orders.

---

## Part 1: Loading the Classification Data

The Archive maintains records of how documents, creatures, and places are classified into hierarchical categories.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('deep')

print("Libraries loaded. The classification records are ready.")

In [ ]:
# Load the Archive Classification records from GitHub
BASE_URL = "https://raw.githubusercontent.com/buildLittleWorlds/densworld-datasets/main/data/"

classifications = pd.read_csv(BASE_URL + "archive_classifications.csv")

print(f"Classification records loaded: {len(classifications):,} entries")
print(f"Type: {type(classifications)}")

Each record represents a classification relationship—an arrow in the hierarchy.

---

## Part 2: Understanding the Classification Schema

In [ ]:
# View the first few classification records
classifications.head(10)

In [ ]:
# List all columns
print("Columns in the Classification records:")
for col in classifications.columns:
    print(f"  - {col}")

Each classification record contains:

| Column | Category Theory Term | Description |
|--------|---------------------|-------------|
| `classification_id` | — | Unique identifier |
| `item_id` | — | What is being classified (document, creature, place) |
| `item_name` | Object | The name of the classified entity |
| `parent_category` | Domain (source) | The higher-level category |
| `child_category` | Codomain (target) | The lower-level category |
| `classification_level` | Depth | 1=root, 2=subcategory, 3=sub-subcategory |
| `classifier` | — | Who made the classification |
| `morphism_exists` | — | Whether the inclusion morphism exists |

The key insight: `parent_category` → `child_category` represents "child is contained in parent", which is a morphism in the pre-order category.

---

## Part 3: Exploring the Hierarchy Structure

Let's understand what classification systems exist in the Archive.

In [ ]:
# What classification systems are represented?
print("Classification systems:")
print(classifications['region_system'].value_counts())

In [ ]:
# What are the root categories? (Level 1)
roots = classifications[classifications['classification_level'] == 1]

print("Root categories (top-level):")
print(roots[['classification_id', 'item_name', 'region_system']])

In [ ]:
# Distribution of classification levels
print("Classification depth distribution:")
level_counts = classifications['classification_level'].value_counts().sort_index()
print(level_counts)

fig, ax = plt.subplots(figsize=(8, 5))
level_counts.plot(kind='bar', ax=ax, color='steelblue', alpha=0.7)
ax.set_xlabel('Classification Level (Depth)')
ax.set_ylabel('Number of Records')
ax.set_title('Depth Distribution in Archive Classification')
ax.set_xticklabels(['Level 1 (Root)', 'Level 2 (Subcategory)', 'Level 3 (Sub-sub)'], rotation=0)
plt.tight_layout()
plt.show()

The Archive has a three-level hierarchy:
- **Level 1**: Root categories (scholarly_works, creatures, places, field_records)
- **Level 2**: Subcategories (predators, aquatic_fauna, mathematical_treatises, etc.)
- **Level 3**: Further refinements (apex_predators, ontological_works, etc.)

---

## Part 4: Building the Pre-order Category

Now let's construct the category explicitly. In a pre-order:
- Objects are the categories and items
- There's a morphism A → B if and only if A is a subcategory of B (or equals B)

We'll focus on the creature taxonomy as a clear example.

In [ ]:
# Filter to the capital taxonomy system
creature_taxonomy = classifications[
    classifications['region_system'] == 'capital_taxonomy'
]

print(f"Creature taxonomy records: {len(creature_taxonomy)}")
print("\nRecords:")
print(creature_taxonomy[['item_name', 'parent_category', 'child_category', 'classification_level']])

In [ ]:
# Build the pre-order relation as a directed graph
# Edge: parent → child means "child is contained in parent"
# (We reverse this to show containment direction: A ≤ B means A → B)

G_taxonomy = nx.DiGraph()

# Add edges from child to parent (child ≤ parent)
for _, row in creature_taxonomy.iterrows():
    if row['child_category'] != row['parent_category']:  # Skip self-loops initially
        G_taxonomy.add_edge(row['child_category'], row['parent_category'])

print(f"Pre-order category constructed:")
print(f"  Objects (nodes): {G_taxonomy.number_of_nodes()}")
print(f"  Direct relations (edges): {G_taxonomy.number_of_edges()}")

In [ ]:
# List all objects in the taxonomy category
print("Objects in the Creature Taxonomy Category:")
for node in sorted(G_taxonomy.nodes()):
    print(f"  • {node}")

### Verifying Pre-order Properties

Let's verify that this is indeed a pre-order category by checking reflexivity and transitivity.

In [ ]:
# 1. REFLEXIVITY: Every object should have an identity (A ≤ A)
# In the graph representation, this means self-loops or implicit identity

print("Checking REFLEXIVITY:")
print("  Every object A has A ≤ A (identity morphism)")
print("  In a pre-order, identity is implicit for all objects.")
print(f"  Objects in category: {list(G_taxonomy.nodes())}")
print("  ✓ Reflexivity holds by definition (every category has identity morphisms)")

In [ ]:
# 2. TRANSITIVITY: If A ≤ B and B ≤ C, then A ≤ C
# This is composition in category theory

# Compute transitive closure to see all implied relations
transitive_closure = nx.transitive_closure(G_taxonomy)

print("Checking TRANSITIVITY:")
print(f"  Direct edges: {G_taxonomy.number_of_edges()}")
print(f"  After transitive closure: {transitive_closure.number_of_edges()}")

# Example: apex_predators ≤ predators and predators ≤ creatures
#          implies apex_predators ≤ creatures
print("\nExample composition:")
print("  apex_predators → predators: ", transitive_closure.has_edge('apex_predators', 'predators'))
print("  predators → creatures: ", transitive_closure.has_edge('predators', 'creatures'))
print("  apex_predators → creatures (composed): ", transitive_closure.has_edge('apex_predators', 'creatures'))

In [ ]:
# 3. ANTISYMMETRY: If A ≤ B and B ≤ A, then A = B
# This would mean no cycles (except self-loops)

print("Checking ANTISYMMETRY (for partial order):")
cycles = list(nx.simple_cycles(G_taxonomy))
if len(cycles) == 0:
    print("  ✓ No cycles found - this is a PARTIAL ORDER (not just pre-order)")
else:
    print(f"  ✗ Found {len(cycles)} cycles - this is only a pre-order")
    for cycle in cycles[:5]:
        print(f"    Cycle: {' → '.join(cycle)}")

The Archive's taxonomy is a **partial order**—a pre-order with antisymmetry. This is typical of well-designed classification systems: nothing can contain something that contains it.

---

## Part 5: Visualizing the Hierarchy

A pre-order category can be visualized as a DAG (Directed Acyclic Graph) when it's also a partial order.

In [ ]:
# Visualize the creature taxonomy as a tree
fig, ax = plt.subplots(figsize=(12, 8))

# Use hierarchical layout
# First reverse edges so layout flows top-down
G_reversed = G_taxonomy.reverse()

try:
    pos = nx.nx_agraph.graphviz_layout(G_reversed, prog='dot')
except:
    # Fallback if graphviz not available
    pos = nx.spring_layout(G_reversed, k=2, iterations=50, seed=42)

# Draw
nx.draw_networkx_nodes(G_reversed, pos, node_size=2500, node_color='lightblue', ax=ax)
nx.draw_networkx_labels(G_reversed, pos, font_size=8, ax=ax)
nx.draw_networkx_edges(G_reversed, pos, edge_color='gray', arrows=True, 
                        arrowsize=15, connectionstyle='arc3,rad=0.1', ax=ax)

ax.set_title('Creature Taxonomy as a Pre-order Category\n(Arrows show "is subcategory of")', fontsize=12)
ax.axis('off')
plt.tight_layout()
plt.show()

### The Full Archive Classification

Let's build a unified view of all classification systems.

In [ ]:
# Build the complete classification pre-order
G_full = nx.DiGraph()

# Add all hierarchical relationships
for _, row in classifications.iterrows():
    child = row['child_category']
    parent = row['parent_category']
    if child != parent:
        G_full.add_edge(child, parent, system=row['region_system'])

print(f"Complete Archive Classification Category:")
print(f"  Objects (categories/items): {G_full.number_of_nodes()}")
print(f"  Direct containment relations: {G_full.number_of_edges()}")

In [ ]:
# Visualize with color-coding by depth
fig, ax = plt.subplots(figsize=(16, 12))

# Reverse for top-down layout
G_full_rev = G_full.reverse()

# Calculate depth for each node
roots = [n for n in G_full_rev.nodes() if G_full_rev.in_degree(n) == 0]

# Color by out-degree (how many subcategories)
node_colors = [G_full.out_degree(n) for n in G_full_rev.nodes()]

try:
    pos = nx.nx_agraph.graphviz_layout(G_full_rev, prog='dot')
except:
    pos = nx.spring_layout(G_full_rev, k=1.5, iterations=50, seed=42)

# Draw
nodes = nx.draw_networkx_nodes(G_full_rev, pos, node_size=1500, 
                                node_color=node_colors, cmap='YlOrRd',
                                alpha=0.8, ax=ax)
nx.draw_networkx_labels(G_full_rev, pos, font_size=6, ax=ax)
nx.draw_networkx_edges(G_full_rev, pos, edge_color='gray', arrows=True,
                        alpha=0.5, arrowsize=10, ax=ax)

plt.colorbar(nodes, ax=ax, label='Number of subcategories')
ax.set_title('Complete Archive Classification\n(The Archive is a Category)', fontsize=14)
ax.axis('off')
plt.tight_layout()
plt.show()

---

## Part 6: The "At Most One Morphism" Property

What makes pre-orders special is that between any two objects, there's at most one morphism.

Let's verify this and contrast with the passage diagrams.

In [ ]:
# In the classification pre-order, check for multiple edges
print("Classification pre-order: Checking for multiple morphisms")

edge_counts = {}
for u, v in G_full.edges():
    key = (u, v)
    edge_counts[key] = edge_counts.get(key, 0) + 1

multiple_edges = {k: v for k, v in edge_counts.items() if v > 1}
if multiple_edges:
    print(f"  Found {len(multiple_edges)} pairs with multiple edges")
else:
    print("  ✓ At most one morphism between any pair of objects")
    print("  → This confirms the pre-order property")

In [ ]:
# Compare with passage diagrams - which CAN have multiple morphisms
passages = pd.read_csv(BASE_URL + "passage_diagrams.csv")

# Count how many morphisms exist between each pair
passage_pairs = passages.groupby(['source_object', 'target_object']).size()
multi_morphism_pairs = passage_pairs[passage_pairs > 1]

print("\nPassage diagrams: Checking for multiple morphisms")
print(f"  Total object pairs with passages: {len(passage_pairs)}")
print(f"  Pairs with MULTIPLE morphisms: {len(multi_morphism_pairs)}")

if len(multi_morphism_pairs) > 0:
    print("\n  Examples of multiple morphisms between same objects:")
    for (src, tgt), count in list(multi_morphism_pairs.items())[:5]:
        print(f"    {src} → {tgt}: {count} different passages")

This is the key difference:

| Category Type | Morphisms between A and B |
|--------------|---------------------------|
| Pre-order | At most 1 (either A ≤ B or not) |
| General category | Any number (multiple paths possible) |

Classification hierarchies are pre-orders because "is contained in" is a yes/no question—there aren't "multiple ways" to be contained.

---

## Part 7: Path Analysis in Pre-orders

In a pre-order, asking "is there a path from A to B?" is the same as asking "is A ≤ B?"

Let's explore reachability in the taxonomy.

In [ ]:
# Build reachability matrix for the creature taxonomy
# Entry [i,j] = 1 if there's a path from category i to category j

nodes = list(G_taxonomy.nodes())
n = len(nodes)
node_to_idx = {node: i for i, node in enumerate(nodes)}

# Compute transitive closure
closure = nx.transitive_closure(G_taxonomy)

# Build matrix
reach_matrix = np.zeros((n, n), dtype=int)
for i, src in enumerate(nodes):
    for j, tgt in enumerate(nodes):
        if src == tgt:  # Reflexive
            reach_matrix[i, j] = 1
        elif closure.has_edge(src, tgt):
            reach_matrix[i, j] = 1

# Visualize
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(reach_matrix, xticklabels=nodes, yticklabels=nodes,
            cmap='Blues', annot=True, fmt='d', cbar_kws={'label': 'Reachable (≤)'},
            ax=ax)
ax.set_xlabel('Target (B)')
ax.set_ylabel('Source (A)')
ax.set_title('Reachability Matrix: A ≤ B in Creature Taxonomy\n(1 = morphism exists, 0 = no morphism)')
plt.tight_layout()
plt.show()

In [ ]:
# Example queries: "Is X ≤ Y?" = "Is X contained in Y?"
print("Pre-order queries: 'Is A ≤ B?'")
print("=" * 50)

queries = [
    ('apex_predators', 'creatures'),
    ('stakdur', 'predators'),  # Note: stakdur is a creature, not a category here
    ('aquatic_fauna', 'predators'),
    ('creatures', 'creatures'),  # Reflexive
    ('predators', 'apex_predators'),  # Reversed direction
]

for a, b in queries:
    if a not in node_to_idx or b not in node_to_idx:
        print(f"  {a} ≤ {b}: (object not in category)")
        continue
    result = reach_matrix[node_to_idx[a], node_to_idx[b]]
    symbol = "✓" if result else "✗"
    print(f"  {a} ≤ {b}: {symbol}")

Notice:
- `apex_predators ≤ creatures`: Yes, through `apex_predators → predators → creatures`
- `predators ≤ apex_predators`: No! Predators is MORE general, not less
- `creatures ≤ creatures`: Yes (reflexivity/identity)

This captures the intuition of hierarchical containment.

---

## Part 8: Items in the Classification

The Archive doesn't just classify categories—it classifies specific documents, creatures, and places.

In [ ]:
# Find specific items (not category definitions)
items = classifications[classifications['item_id'].str.startswith(('DOC-', 'CR-', 'PL-'))]

print(f"Specific items classified: {len(items)}")
print("\nItems by type:")
print(items.groupby(items['item_id'].str[:3]).size())

In [ ]:
# Show document classifications
docs = items[items['item_id'].str.startswith('DOC-')]
print("Documents and their classifications:")
print(docs[['item_name', 'child_category', 'parent_category', 'classification_level']])

In [ ]:
# Trace: Where does "Patterns of Passage" live in the hierarchy?
patterns = items[items['item_name'] == 'Patterns of Passage']

print("Classification chain for 'Patterns of Passage':")
for _, row in patterns.sort_values('classification_level').iterrows():
    indent = "  " * row['classification_level']
    print(f"{indent}Level {row['classification_level']}: {row['child_category']} → {row['parent_category']}")

Vold's "Patterns of Passage" sits in a multi-level classification:

```
Patterns of Passage
    ↓ (is instance of)
ontological_works
    ↓ (is subcategory of)
philosophical_treatises
    ↓ (is subcategory of)
scholarly_works
    ↓ (is subcategory of)
root
```

This chain of morphisms is a composition in the pre-order category.

---

## Part 9: The ML Connection

Pre-orders appear throughout machine learning:

### Taxonomy Learning
When training models on hierarchical labels (ImageNet, WordNet), the label hierarchy is a pre-order category.

### Ontology Embeddings  
Knowledge graph embeddings must respect the pre-order structure: if A ≤ B, then the embedding should reflect this (e.g., hyperboloid embeddings).

### Decision Trees
The path from root to leaf in a decision tree forms a pre-order category.

### Feature Hierarchies
CNN feature maps form a hierarchy: low-level features (edges) are subcategories of high-level features (objects).

In [ ]:
# Demonstrate: Hierarchical loss weighting
# In multi-label classification, misclassifying as a sibling is less bad than as a distant category

def hierarchy_distance(a, b, closure):
    """Distance in the hierarchy based on pre-order structure."""
    if a == b:
        return 0
    elif closure.has_edge(a, b) or closure.has_edge(b, a):
        # One is ancestor of the other - count path length
        if closure.has_edge(a, b):
            path = nx.shortest_path(closure, a, b)
        else:
            path = nx.shortest_path(closure, b, a)
        return len(path) - 1
    else:
        # Find common ancestor
        return float('inf')  # Simplified: no common structure

# Example: Distance between categories in creature taxonomy
print("Hierarchy distances (for loss weighting):")
pairs = [
    ('apex_predators', 'pack_hunters'),  # Siblings
    ('apex_predators', 'predators'),     # Parent
    ('apex_predators', 'creatures'),     # Grandparent
]

closure = nx.transitive_closure(G_taxonomy)
for a, b in pairs:
    d = hierarchy_distance(a, b, closure)
    print(f"  d({a}, {b}) = {d}")

---

## Exercises

### Exercise 1: Geographic Hierarchy

Filter to the `geographic_registry` system and build its pre-order graph. How many levels does it have?

In [ ]:
# Your code here
# Hint: geo = classifications[classifications['region_system'] == 'geographic_registry']

### Exercise 2: Multiple Classifications

Some items appear at multiple classification levels (e.g., Stakdur at level 2 and level 3). Find all items with multiple classifications and explain what this means categorically.

In [ ]:
# Your code here
# Hint: items.groupby('item_name')['classification_level'].nunique()

### Exercise 3: Classifier Analysis

Who made the most classifications? Is there a pattern in which classifiers handle which types of items?

In [ ]:
# Your code here
# Hint: classifications.groupby('classifier')['item_name'].count()

### Exercise 4: Longest Chain

Find the longest chain of containment relationships in the full classification system. This represents the "deepest" classification.

In [ ]:
# Your code here
# Hint: Use nx.dag_longest_path(G_full) if the graph is a DAG

---

## Discussion Questions

1. The Archive treats "anomalous_entities" as a category within creatures. But Vold might argue that anomalies defy categorical containment. Is there a categorical notion that captures "resisting classification"?

2. In a pre-order category, composition is trivial (if both morphisms exist, so does the composite). What does this say about the relationship between pre-orders and general categories?

3. Machine learning taxonomies (like ImageNet) often have "multiple inheritance" (a dog can be both a pet and a mammal). How would you represent this categorically? Is it still a pre-order?

---

## Summary

In this tutorial, you learned:

| Concept | What You Learned |
|---------|------------------|
| Pre-order | A category with at most one morphism between any pair of objects |
| Reflexivity | Every object has A ≤ A (identity morphism) |
| Transitivity | If A ≤ B and B ≤ C, then A ≤ C (composition) |
| Partial order | Pre-order + antisymmetry (no cycles except identity) |
| Classification hierarchy | Natural example of a pre-order/partial order category |

| Skill | Code Pattern |
|-------|--------------|
| Build hierarchy graph | `G.add_edge(child, parent)` |
| Compute transitive closure | `nx.transitive_closure(G)` |
| Detect cycles | `nx.simple_cycles(G)` |
| Reachability query | `closure.has_edge(a, b)` |
| Visualize hierarchy | `nx.draw_networkx_*()` with hierarchical layout |

---

## Next Tutorial

In **Tutorial 3: Composition and Identity**, you will dive deeper into the fundamental laws that make a category:

- The **identity axiom**: f ∘ id = f = id ∘ f
- The **associativity axiom**: (f ∘ g) ∘ h = f ∘ (g ∘ h)
- How to verify these laws in data
- Composition chains in lifecycle and intellectual passages

> *"Composition is the heart of categorical thinking. To compose is to recognize that the intermediate step can be forgotten—only the beginning and end matter, yet the structure remains."*
> — Tessery Vold, "Patterns of Passage," Year 893

---

## Credits

**Source Material:** Tai-Danae Bradley, "Category Theory and Language Models" (Cartesian Cafe)

**Densworld Integration:** The Relational Foundations course applies categorical concepts through the framework of Tessery Vold.

**Learn more:** [buildLittleWorlds](https://github.com/buildLittleWorlds)

---

> *"The Archive itself is a category. Every classification is a morphism. To place a document is to draw an arrow. But unlike the passages of creatures, these arrows are unique—or absent. This constraint is not a limitation. It is the structure that makes retrieval possible."*
> — Tessery Vold, on discovering categorical structure in archival practice